In [1]:
import torch
import transformers
import librosa
import keras
import cv2

In [2]:
from text_model import predict_text_emotion

In [3]:
from speech_model import predict_speech_emotion

In [4]:
from image_model import predict_image_emotion

In [5]:
import numpy as np

emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def fuse_emotions(text_probs, speech_probs, image_probs, weights=(0.5, 0.25, 0.25)):
    text_weight, speech_weight, image_weight = weights
    combined = (
        text_weight * text_probs +
        speech_weight * speech_probs +
        image_weight * image_probs
    )
    final_index = np.argmax(combined)
    return emotion_labels[final_index], combined


In [11]:
from image_model import predict_image_emotion

image_path = "data/sample_image.jpg"
probs = predict_image_emotion(image_path)
print("Prediction probability distribution:", probs)

1/1 [==============================] - 0s 86ms/step
Prediction probability distribution: [6.9470567e-05 8.3944516e-04 1.0972669e-03 3.1081665e-05 1.7236372e-03
 9.9618918e-01 4.9976585e-05]


In [10]:
# Sample input
text_input = "I feel very frustrated and alone."
audio_input = "data/sample_audio.wav"
image_input = "data/sample_image.jpg"

text_probs = predict_text_emotion(text_input)
image_probs = predict_image_emotion(image_input)
speech_probs = predict_speech_emotion(audio_input)


final_emotion, prob_vector = fuse_emotions(text_probs, speech_probs, image_probs)

print("Predicted Emotion:", final_emotion)
print("Probabilities:", prob_vector)


1/1 [==============================] - 0s 132ms/step
Predicted Emotion: angry
Probabilities: [0.50168648 0.01180454 0.01430752 0.01936726 0.17103935 0.26862427
 0.0131706 ]


In [20]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("test_data/fusion_text_label_dataset.csv")
df.head()

,text,image_path,audio_path,label
0,I'm really mad right now.,test_data/images/test_0017_aligned.jpg,test_data/audio/03-01-05-01-01-01-01.wav,angry
1,This is so frustrating!,test_data/images/test_0027_aligned.jpg,test_data/audio/03-01-05-01-01-02-01.wav,angry
2,I'm really mad right now.,test_data/images/test_0037_aligned.jpg,test_data/audio/03-01-05-01-02-01-01.wav,angry
3,This is so frustrating!,test_data/images/test_0042_aligned.jpg,test_data/audio/03-01-05-01-02-02-01.wav,angry
4,Why does this always happen to me?,test_data/images/test_0057_aligned.jpg,test_data/audio/03-01-05-02-01-01-01.wav,angry


In [19]:
import os
import cv2

# 看工作目录
print("Current Working Directory:", os.getcwd())

# 看文件是否真的存在
path = "images/test_0017_aligned.jpg"
print("Exists:", os.path.exists(path))

# 尝试读取图片
img = cv2.imread(path)
if img is None:
    print("💥 OpenCV could not read the image.")
else:
    print("✅ Image read successfully:", img.shape)


Current Working Directory: C:\Users\YANG LY\engineering team project
Exists: False
💥 OpenCV could not read the image.


In [21]:
correct = 0
total = 0
wrong_samples = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text_probs = predict_text_emotion(row['text'])
        image_probs = predict_image_emotion(row['image_path'])
        speech_probs = predict_speech_emotion(row['audio_path'])
        pred_emotion, _ = fuse_emotions(text_probs, speech_probs, image_probs)

        if pred_emotion == row['label']:
            correct += 1
        else:
            wrong_samples.append({
                'text': row['text'],
                'image': row['image_path'],
                'audio': row['audio_path'],
                'true': row['label'],
                'pred': pred_emotion
            })
        total += 1
    except Exception as e:
        print(f"Error at row {idx}: {e}")
        continue

accuracy = correct / total
print(f"✅ Fusion Model Accuracy: {accuracy:.4f} ({correct}/{total})")

  0%|                                                                                          | 0/140 [00:00<?, ?it/s]

1/1 [==============================] - 0s 442ms/step


  1%|▌                                                                                 | 1/140 [00:01<02:25,  1.05s/it]

1/1 [==============================] - 0s 104ms/step


  1%|█▏                                                                                | 2/140 [00:01<01:32,  1.50it/s]

...

1/1 [==============================] - 0s 74ms/step


 99%|███████████████████████████████████████████████████████████████████████████████▍| 139/140 [00:53<00:00,  2.84it/s]

1/1 [==============================] - 0s 86ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 140/140 [00:53<00:00,  2.62it/s]

✅ Fusion Model Accuracy: 0.5357 (75/140)


In [24]:
from sklearn.metrics import classification_report

true_labels = df['label'].tolist()
pred_labels = []

for idx, row in df.iterrows():
    try:
        t = predict_text_emotion(row['text'])
        i = predict_image_emotion(row['image_path'])
        s = predict_speech_emotion(row['audio_path'])
        pred, _ = fuse_emotions(t, s, i)
        pred_labels.append(pred)
    except:
        pred_labels.append("error")  # 或者跳过

# 打印每类表现
print(classification_report(true_labels, pred_labels))

1/1 [==============================] - 0s 76ms/step
              precision    recall  f1-score   support

       angry       0.41      1.00      0.58        20
     disgust       1.00      0.05      0.10        20
        fear       1.00      0.75      0.86        20
       happy       0.39      1.00      0.56        20
     neutral       0.00      0.00      0.00        20
         sad       0.95      0.90      0.92        20
    surprise       1.00      0.05      0.10        20

    accuracy                           0.54       140
   macro avg       0.68      0.54      0.44       140
weighted avg       0.68      0.54      0.44       140

